In [18]:
import numpy as np

print(f"numpy version: {np.__version__}")

s = np.random.binomial(1, 0.5)

if (s):
    print("yes")
else:
    print("no")

if np.random.binomial(1, 0.5):
    print("yes (2)")
else:
    print("no (2)")

numpy version: 1.24.3
no
yes (2)


In [21]:
import moviepy

print(f"moviepy version: {moviepy.__version__}")

moviepy version: 1.0.3


In [22]:
import tensorflow as tf

print(f"tensorflow version: {tf.__version__}")

tensorflow version: 2.16.1


In [36]:
states = tf.convert_to_tensor([[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]], dtype=tf.float32)
actions = tf.convert_to_tensor([1, 0, 1], dtype=tf.int64)
rewards = tf.convert_to_tensor([0.5, 0.4, 0.6], dtype=tf.float32)
next_states = tf.convert_to_tensor([[1.5, 2.5], [2.5, 3.5], [3.5, 4.5]], dtype=tf.float32)
is_done = tf.convert_to_tensor([0, 0, 1], dtype=tf.uint8)


In [49]:
predicted_qvalues = tf.convert_to_tensor([[0.75, 0.1], [0.25, 0.5], [0.4, 0.6]], dtype=tf.float32)
print(f"predicted_qvalues:\n {predicted_qvalues}")

one_hot = tf.one_hot(actions, 2)
print(f"one_hot:\n {one_hot}")

predicted_qvalues_for_actions = tf.reduce_sum(predicted_qvalues * one_hot, axis=-1)
print(f"predicted_qvalues_for_actions:\n {predicted_qvalues_for_actions}")

predicted_next_qvalues = tf.convert_to_tensor([[0.45, 0.25], [0.33, 0.44], [0.28, 0.74]], dtype=tf.float32)
print(f"predicted_next_qvalues:\n {predicted_next_qvalues}")

next_state_values = tf.reduce_max(predicted_next_qvalues, axis=-1)
print(f"next_state_values:\n {next_state_values}")

predicted_qvalues:
 [[0.75 0.1 ]
 [0.25 0.5 ]
 [0.4  0.6 ]]
one_hot:
 [[0. 1.]
 [1. 0.]
 [0. 1.]]
predicted_qvalues_for_actions:
 [0.1  0.25 0.6 ]
predicted_next_qvalues:
 [[0.45 0.25]
 [0.33 0.44]
 [0.28 0.74]]
next_state_values:
 [0.45 0.44 0.74]


In [5]:
import numpy as np

np.random.binomial(1, 0.5)

0

In [11]:
if 0:
    print("0")
else:
    print("not 0")
    
if 1:
    print("1")
else:
    print("not 1")

if 2:
    print("2")
else:
    print("not 2")

not 0
1
2


In [15]:
import tensorflow as tf
import keras

print(f"tensorflow version: {tf.__version__}")
print(f"keras version: {keras.__version__}")


tensorflow version: 2.16.1
keras version: 3.3.3


In [18]:
nn_model = keras.models.Sequential([
    keras.layers.Input(shape=(2,)),
    keras.layers.Dense(1, activation='sigmoid',
                 kernel_initializer=keras.initializers.RandomNormal(stddev=0.05),
                 bias_initializer=keras.initializers.RandomNormal(stddev=0.05))
    ])

print(f"weights: {nn_model.trainable_weights}")

weights: [<KerasVariable shape=(2, 1), dtype=float32, path=sequential/dense/kernel>, <KerasVariable shape=(1,), dtype=float32, path=sequential/dense/bias>]


In [19]:
nn_model.trainable_weights[0]


<KerasVariable shape=(2, 1), dtype=float32, path=sequential/dense/kernel>

In [20]:
nn_model.trainable_weights[1]

<KerasVariable shape=(1,), dtype=float32, path=sequential/dense/bias>

In [22]:
eligibility_traces = [tf.zeros(weights.shape) for weights in nn_model.trainable_weights]

print(f"eligibility_traces: {eligibility_traces}")

eligibility_traces: [<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>]


In [23]:
state = tf.Variable([[1.0, 2.0]])
print(state)

<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[1., 2.]], dtype=float32)>


In [38]:
nn_model(state)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.5182955]], dtype=float32)>

In [25]:
expected_value = [[0.6]]

In [35]:
with tf.GradientTape() as tape:
    predicted_value = nn_model(state)
    gradients = tape.gradient(predicted_value, nn_model.trainable_weights)

print(gradients)
print(nn_model.get_weights())

[<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.24966528],
       [0.49933055]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.24966528], dtype=float32)>]
[array([[-0.03440512],
       [ 0.0381441 ]], dtype=float32), array([0.03133164], dtype=float32)]


In [49]:
print("BEFORE:")
print(nn_model.get_weights())

predicted_value = nn_model(state)
print(f"PREDICTED: {predicted_value}")
expected_value = tf.Variable([[0.6]])
print(f"EXPECTED: {expected_value}")

output_dim = 1
lamda = 0.7
learning_rate = 0.01

for i, gradient in enumerate(gradients):
    eligibility_traces[i] = lamda * eligibility_traces[i] + gradient
    weight = nn_model.trainable_weights[i]
    weight.assign_add(learning_rate * tf.reshape(expected_value - predicted_value, shape=(output_dim,)) * eligibility_traces[i])

print("AFTER:")
print(nn_model.get_weights())

print("ELIGIBILITY TRACES:")
print(eligibility_traces)


BEFORE:
[array([[-0.03227917],
       [ 0.04239601]], dtype=float32), array([0.03227585], dtype=float32)]
PREDICTED: [[0.52118444]]
EXPECTED: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[0.6]], dtype=float32)>
AFTER:
[array([[-0.03166107],
       [ 0.04363221]], dtype=float32), array([0.03282153], dtype=float32)]
ELIGIBILITY TRACES:
[<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.78424186],
       [1.5684837 ]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.69234675], dtype=float32)>]


In [50]:
predicted_value = nn_model(state)
print(f"PREDICTED: {predicted_value}")

PREDICTED: [[0.52209187]]


In [51]:
state_2 = tf.Variable([[1.5, 2.5]])

print("BEFORE:")
print(nn_model.get_weights())

predicted_value = nn_model(state_2)
print(f"PREDICTED: {predicted_value}")
expected_value = tf.Variable([[0.7]])
print(f"EXPECTED: {expected_value}")

output_dim = 1
lamda = 0.7
learning_rate = 0.01

for i, gradient in enumerate(gradients):
    eligibility_traces[i] = lamda * eligibility_traces[i] + gradient
    weight = nn_model.trainable_weights[i]
    weight.assign_add(learning_rate * tf.reshape(expected_value - predicted_value, shape=(output_dim,)) * eligibility_traces[i])

print("AFTER:")
print(nn_model.get_weights())

print("ELIGIBILITY TRACES:")
print(eligibility_traces)

BEFORE:
[array([[-0.03166107],
       [ 0.04363221]], dtype=float32), array([0.03282153], dtype=float32)]
PREDICTED: [[0.5235851]]
EXPECTED: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[0.7]], dtype=float32)>
AFTER:
[array([[-0.03025216],
       [ 0.04645003]], dtype=float32), array([0.03411696], dtype=float32)]
ELIGIBILITY TRACES:
[<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.7986345],
       [1.597269 ]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.734308], dtype=float32)>]


In [52]:
nn_model(state_2)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.52619195]], dtype=float32)>

In [53]:
print(nn_model.get_weights())

[array([[-0.03025216],
       [ 0.04645003]], dtype=float32), array([0.03411696], dtype=float32)]


In [54]:
state_3 = tf.Variable([[1.0, 1.0]])
nn_model(state_2)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.52619195]], dtype=float32)>

In [55]:
%load_ext tensorboard


In [60]:
expected_value = [[1.0]]
predicted_value = tf.Variable([[0.54]])

expected_value - predicted_value

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.45999998]], dtype=float32)>